In [5]:
# -*- coding: utf-8 -*-
# ==================================================================================================
# CDR — End-to-end build (REMOTE ONLY via WebDAV)
# - Reads Qualtrics CSV + QSF from Research Drive
# - Enriches item labels from QSF
# - Scores digiQ (True/False with DK + N/A handling)
# - Drops ONLY attention-check failures
# - Builds CDR composites (env/soc/gov/total)
# - Builds RQ1 codebook sections
# - Demographics mapping + age-band normalization (NL labels)
# - Pretty-prints Markdown codebooks & a summary to Research Drive
# ==================================================================================================

# !pip install pandas numpy pyarrow

import io
import json
import re
import unicodedata
import posixpath

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

# ---- WebDAV I/O helpers (your rd_utils wrapped by rd_io) ----
from rd_io import (
    BASE_DIR, DATA_DIR, OUT_DIR, rd_join, rd_parent,
    rd_download_bytes, rd_upload_bytes,
    rd_read_text, rd_write_text, rd_write_markdown, rd_write_json,
    rd_read_csv_df, rd_write_csv_df,
    rd_read_parquet_df, rd_write_parquet_df,
)

# --------------------------------------------------------------------------------------------------
# CONFIG (REMOTE PATHS)
# --------------------------------------------------------------------------------------------------
CSV_PATH = rd_join(DATA_DIR, "COS_AI-Tech_October 6, 2025_05.00.csv")  # remote
QSF_PATH = rd_join(DATA_DIR, "COS_AI-Tech.qsf")                         # remote

OUT_KEEP_DATA     = rd_join(OUT_DIR, "cdr_keep_after_attn.parquet")
OUT_DROPPED_ATTN  = rd_join(OUT_DIR, "dropped_due_to_attention_checks.csv")
OUT_THEORY_MAP    = rd_join(OUT_DIR, "cdr_theory_map.csv")

OUT_CODEBOOK_PRED_MD  = rd_join(OUT_DIR, "codebook_predictors.md")
OUT_CODEBOOK_DIGIQ_MD = rd_join(OUT_DIR, "codebook_digiq_key.md")
OUT_CODEBOOK_CDR_MD   = rd_join(OUT_DIR, "codebook_cdr_items.md")
OUT_CODEBOOK_RQ1_MD   = rd_join(OUT_DIR, "codebook_rq1_attribs.md")
OUT_CODEBOOK_DEMOG_MD = rd_join(OUT_DIR, "codebook_demographics.md")

# Column groups (export header names)
ENV_CONSC = [f"CDR pred, envconsc_{i}" for i in range(1, 5)]
SOC_CONSC = [f"CDR pred, socconsc_{i}" for i in range(1, 4)]
TRUST     = [f"CDR pred, trust_{i}"    for i in range(1, 6)]
DIGIQ     = [f"CDR pred, digiQ_{i}"    for i in range(1, 9)]
CDR_20    = [f"CDR attributions _{i}"  for i in range(1, 21)]

RQ1_ENV   = [f"responsibility attri_{i}"    for i in range(1, 9)]
RQ1_SOC   = [f"resp_attribut_societ_{i}"    for i in range(1, 9)]
RQ1_GOV   = [f"resp_attribut_regula_{i}"    for i in range(1, 9)]

ATTN1 = "AttentionCheck1"
ATTN2 = "AttentionCheck2"

# DigiQ answer key (confirmed)
DIGIQ_KEY = {f"CDR pred, digiQ_{i}": ("juist" if i >= 2 else "fout") for i in range(1, 9)}
SYN_CORRECT_FALSE = {"fout", "onjuist", "niet waar", "false", "incorrect"}
SYN_CORRECT_TRUE  = {"juist", "waar", "true", "correct"}
SYN_DK = {"ik weet het niet", "weet ik niet", "dont know", "dk"}
SYN_NA = {"ik begrijp de vraag niet", "ik begrijp de vraag niet.", "nvt", "n.v.t.", "n/a", "na"}

# Regex to catch {"ImportId":"QIDxx_yy"} labels in the CSV's 2nd row
_importid_re = re.compile(r'^\s*\{\s*"ImportId"\s*:\s*"(QID\d+)(?:_(\d+))?"\s*\}\s*$')

# --------------------------------------------------------------------------------------------------
# Helpers
# --------------------------------------------------------------------------------------------------
def hrule(title):
    print("\n" + "-" * 100)
    print(f"# {title}")
    print("-" * 100 + "\n")

def to_markdown_table(rows, header):
    df = pd.DataFrame(rows, columns=header)
    def esc(x):
        if pd.isna(x): return ""
        return str(x).replace("|", r"\|").replace("\n", "<br />")
    df = df.applymap(esc)
    out = ["|" + "|".join(df.columns) + "|",
           "|" + "|".join([":" + "-"*(max(3, len(c))) for c in df.columns]) + "|"]
    for _, row in df.iterrows():
        out.append("|" + "|".join(row.values) + "|")
    return "\n".join(out)

def extract_first_int(s):
    if pd.isna(s): return np.nan
    if isinstance(s, (int, float, np.integer, np.floating)): return float(s)
    m = re.search(r'[-+]?\d+', str(s))
    return float(m.group(0)) if m else np.nan

def _normtxt(s: str) -> str:
    if not s: return ""
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    s = s.strip().lower()
    s = s.replace("’","'").replace("‘","'").replace("“","\"").replace("”","\"")
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

def basename(p: str) -> str:
    return posixpath.basename(p)

# --------------------------------------------------------------------------------------------------
# 1) READ DATA (REMOTE)
# --------------------------------------------------------------------------------------------------
def read_qualtrics_csv(remote_csv_path: str) -> pd.DataFrame:
    df = rd_read_csv_df(remote_csv_path, low_memory=False)
    # Heuristic: drop Qualtrics "label row" if present
    if df.iloc[0].astype(str).str.contains("Geef aan of je instemt", case=False, na=False).any():
        df = df.iloc[1:].reset_index(drop=True)
    return df

def read_csv_header_and_label(remote_csv_path: str) -> pd.DataFrame:
    raw = rd_download_bytes(remote_csv_path)
    return pd.read_csv(io.BytesIO(raw), nrows=2, header=None, low_memory=False)

d_raw = read_qualtrics_csv(CSV_PATH)
df_labels = read_csv_header_and_label(CSV_PATH)
csv_header = df_labels.iloc[0].tolist()
csv_label_row = df_labels.iloc[1].tolist() if df_labels.shape[0] > 1 else [""] * len(csv_header)

d = d_raw.copy()
d.columns = csv_header

# --------------------------------------------------------------------------------------------------
# 2) READ QSF (REMOTE) & ENRICH LABELS
# --------------------------------------------------------------------------------------------------
def load_qsf(remote_qsf_path: str):
    qsf = json.loads(rd_read_text(remote_qsf_path, encoding="utf-8"))
    out = {}
    for el in qsf.get("SurveyElements", []):
        if el.get("Element") == "SQ":
            out[el["PrimaryAttribute"]] = el.get("Payload", {})
    return out

qid_payload = load_qsf(QSF_PATH)

def enrich_label_with_qsf(qid_payload, label_text):
    if not label_text or not isinstance(label_text, str):
        return label_text
    m = _importid_re.match(label_text.strip())
    if not m:
        return label_text
    qid, sub = m.group(1), m.group(2)
    payload = qid_payload.get(qid)
    if not payload:
        return label_text
    qtext = (payload.get("QuestionText") or "").strip()
    subtext = None
    if sub and "SubQuestions" in payload:
        sq = payload["SubQuestions"].get(sub)
        if isinstance(sq, dict):
            subtext = (sq.get("ChoiceText") or sq.get("Display") or "").strip()
    if not subtext and sub and "Choices" in payload:
        ch = payload["Choices"].get(sub)
        if isinstance(ch, dict):
            subtext = (ch.get("ChoiceText") or ch.get("Display") or "").strip()
    if qtext and subtext:
        return f"{qtext} — [{subtext}]"
    return qtext or label_text

enriched_label_map = {
    col: enrich_label_with_qsf(qid_payload, str(lab))
    for col, lab in zip(csv_header, csv_label_row)
}

# --------------------------------------------------------------------------------------------------
# 3) ATTENTION FILTER (ONLY)
# --------------------------------------------------------------------------------------------------
def parse_attn1(x):
    s = str(x).strip().lower()
    return "geel" in s or s == "3"

def parse_attn2(x):
    s = str(x).strip().lower()
    return s == "3"

attn1_ok = d[ATTN1].apply(parse_attn1) if ATTN1 in d.columns else pd.Series(False, index=d.index)
attn2_ok = d[ATTN2].apply(parse_attn2) if ATTN2 in d.columns else pd.Series(False, index=d.index)
attn_pass = attn1_ok & attn2_ok
idx_drop = d.index[~attn_pass].tolist()

# Save dropped cases (REMOTE)
drop_report = pd.DataFrame({
    "index": idx_drop,
    "ResponseId": d.get("ResponseId", pd.Series(index=d.index, dtype=object)).reindex(idx_drop).values,
    "AttentionCheck1": d.get(ATTN1, pd.Series(index=d.index, dtype=object)).reindex(idx_drop).values,
    "AttentionCheck2": d.get(ATTN2, pd.Series(index=d.index, dtype=object)).reindex(idx_drop).values,
})
rd_write_csv_df(drop_report, OUT_DROPPED_ATTN, index=False, encoding="utf-8")

d_keep = d.loc[attn_pass].copy()

# Coerce Likert-like to numeric (only declared groups)
LIKERT_CANDIDATES = set(ENV_CONSC + SOC_CONSC + TRUST + CDR_20 + RQ1_ENV + RQ1_SOC + RQ1_GOV)
for col in LIKERT_CANDIDATES:
    if col in d_keep.columns:
        d_keep[col] = d_keep[col].apply(extract_first_int)

# --------------------------------------------------------------------------------------------------
# 4) DIGIQ — scoring
# --------------------------------------------------------------------------------------------------
def _norm(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().lower().replace(".", "")
    return s

def score_digiq_series(series, correct_key):
    x = series.map(_norm)
    correct_set = SYN_CORRECT_TRUE if correct_key == "juist" else SYN_CORRECT_FALSE
    out = []
    for v in x:
        if pd.isna(v):
            out.append(np.nan)
        elif v in SYN_NA:
            out.append(np.nan)
        elif v in SYN_DK:
            out.append(0.0)
        elif v in correct_set:
            out.append(1.0)
        elif (v in SYN_CORRECT_FALSE) or (v in SYN_CORRECT_TRUE):
            out.append(0.0)
        else:
            out.append(np.nan)
    return pd.Series(out, index=series.index, dtype="float")

for col, key in DIGIQ_KEY.items():
    if col in d_keep.columns:
        d_keep[col + "_sc"] = score_digiq_series(d_keep[col], key)

digiq_sc_cols = [c + "_sc" for c in DIGIQ if (c + "_sc") in d_keep.columns]
if digiq_sc_cols:
    d_keep["digiQ_total"] = d_keep[digiq_sc_cols].sum(axis=1, skipna=True)
    d_keep["digiQ_prop"]  = d_keep[digiq_sc_cols].mean(axis=1, skipna=True)

# --------------------------------------------------------------------------------------------------
# 5) CDR composites (Env/Soc/Gov)
# --------------------------------------------------------------------------------------------------
env_cols = [f"CDR attributions _{i}" for i in range(1, 4)  if f"CDR attributions _{i}" in d_keep.columns]  # 1–3
soc_cols = [f"CDR attributions _{i}" for i in range(4, 13) if f"CDR attributions _{i}" in d_keep.columns]  # 4–12
gov_cols = [f"CDR attributions _{i}" for i in range(13, 21)if f"CDR attributions _{i}" in d_keep.columns]  # 13–20

def _to_numeric(df, cols):
    return df[cols].apply(pd.to_numeric, errors="coerce") if cols else pd.DataFrame(index=df.index)

if env_cols:
    d_keep["CDR_env_mean"] = _to_numeric(d_keep, env_cols).mean(axis=1, skipna=True)
if soc_cols:
    d_keep["CDR_soc_mean"] = _to_numeric(d_keep, soc_cols).mean(axis=1, skipna=True)
if gov_cols:
    d_keep["CDR_gov_mean"] = _to_numeric(d_keep, gov_cols).mean(axis=1, skipna=True)

present_cdr_cols = [c for c in CDR_20 if c in d_keep.columns]
if present_cdr_cols:
    d_keep["CDR_total_mean"] = _to_numeric(d_keep, present_cdr_cols).mean(axis=1, skipna=True)

# --------------------------------------------------------------------------------------------------
# 6) Theory map (variable -> domain) — REMOTE write
# --------------------------------------------------------------------------------------------------
theory_rows = []
for c in env_cols: theory_rows.append([c, "Environmental"])
for c in soc_cols: theory_rows.append([c, "Social"])
for c in gov_cols: theory_rows.append([c, "Governance"])
rd_write_csv_df(pd.DataFrame(theory_rows, columns=["variable", "domain"]), OUT_THEORY_MAP, index=False, encoding="utf-8")

# --------------------------------------------------------------------------------------------------
# 6.5) DEMOGRAPHICS — robust mapping + age normalization
# --------------------------------------------------------------------------------------------------
header_norm_map = { _normtxt(h): h for h in csv_header }

def _col_like(*keys):
    keys_norm = [_normtxt(k) for k in keys]
    for k_norm, col in header_norm_map.items():
        if all(kk in k_norm for kk in keys_norm):
            return col
    return None

col_age   = _col_like("age") or ("Age " if "Age " in d_keep.columns else None)
col_gender= _col_like("gender")
col_empl  = _col_like("work","status") or ("Work_status" if "Work_status" in d_keep.columns else None)
col_edu   = _col_like("education")
col_pol   = _col_like("pol","orient") or ("Pol_orientation" if "Pol_orientation" in d_keep.columns else None)

demog_rows = []

# --- Age: ordered bands ---
if col_age and col_age in d_keep.columns:
    d_keep["dem_age_band_raw"] = d_keep[col_age]

    def _norm_age_band(x):
        if pd.isna(x): return np.nan
        s = str(x).strip()
        mapping = {
            "18 - 24 jaar":"18–24","18-24 jaar":"18–24","18 – 24 jaar":"18–24","18–24 jaar":"18–24","18 - 24":"18–24",
            "25 - 34 jaar":"25–34","25-34 jaar":"25–34","25 – 34 jaar":"25–34","25–34 jaar":"25–34","25 - 34":"25–34",
            "35 - 44 jaar":"35–44","35-44 jaar":"35–44","35 – 44 jaar":"35–44","35–44 jaar":"35–44","35 - 44":"35–44",
            "45 - 54 jaar":"45–54","45-54 jaar":"45–54","45 – 54 jaar":"45–54","45–54 jaar":"45–54","45 - 54":"45–54",
            "55 - 65 jaar":"55–65","55-65 jaar":"55–65","55 – 65 jaar":"55–65","55–65 jaar":"55–65","55 - 65":"55–65",
        }
        if s in mapping: return mapping[s]
        nums = re.findall(r"\d+", s)
        if len(nums) >= 2:
            a, b = nums[0], nums[1]
            return f"{a}–{b}"
        return np.nan

    bands = d_keep[col_age].map(_norm_age_band)
    allowed = ["18–24","25–34","35–44","45–54","55–65"]
    bands = bands.where(bands.isin(allowed))
    d_keep["dem_age_band"] = pd.Categorical(bands, categories=allowed, ordered=True)

    demog_rows.append(["dem_age_band", "", col_age, "Age", "Leeftijdscategorie (geordend: 18–24 … 55–65)"])

# --- Gender ---
if col_gender and col_gender in d_keep.columns:
    d_keep["dem_gender_raw"] = d_keep[col_gender]
    d_keep["dem_gender"] = d_keep[col_gender].astype("category")
    demog_rows.append(["dem_gender", "", col_gender, "Gender", "Geslacht"])

# --- Employment ---
if col_empl and col_empl in d_keep.columns:
    d_keep["dem_employment_raw"] = d_keep[col_empl]
    d_keep["dem_employment"] = d_keep[col_empl].astype("category")
    demog_rows.append(["dem_employment", "", col_empl, "Work status", "Werkstatus"])

# --- Education ---
if col_edu and col_edu in d_keep.columns:
    d_keep["dem_education_raw"] = d_keep[col_edu]
    d_keep["dem_education"] = d_keep[col_edu].astype("category")
    demog_rows.append(["dem_education", "", col_edu, "Education", "Hoogst voltooide opleiding"])

# --- Political L–R (numeric) ---
if col_pol and col_pol in d_keep.columns:
    d_keep["dem_political_lr"] = d_keep[col_pol].apply(extract_first_int)
    demog_rows.append(["dem_political_lr", "", col_pol, "Political L–R", "Politieke zelfplaatsing (1–10)"])

# Build DEMOGRAPHICS codebook (REMOTE)
if demog_rows:
    demog_md = "\n\n# Codebook: Demographics\n" + to_markdown_table(
        demog_rows,
        ["Afgeleide variabele", "QID (bron)", "CSV-kolom", "Vraag (bron)", "Beschrijving"]
    )
else:
    demog_md = "\n\n# Codebook: Demographics\n_Geen demografische variabelen gevonden._"

rd_write_markdown(demog_md, OUT_CODEBOOK_DEMOG_MD)

# --------------------------------------------------------------------------------------------------
# 7) Markdown codebooks (Predictors / DigiQ / CDR / RQ1) — REMOTE
# --------------------------------------------------------------------------------------------------
def md_section(title): return f"\n\n# {title}\n"

def md_table_from_columns(cols, extra_cols=None):
    rows = []
    for c in cols:
        rows.append([c, enriched_label_map.get(c, "")] + ([extra_cols.get(c, "")] if extra_cols else []))
    header = ["Variabele", "Volledige NL-vraag (QSF-verrijkt of CSV)"]
    if extra_cols: header.append("Opmerking/Key")
    return to_markdown_table(rows, header)

# Predictors
pred_cols = [c for c in (ENV_CONSC + SOC_CONSC + TRUST) if c in d.columns]
pred_md = md_section("Codebook: Predictors") + md_table_from_columns(pred_cols)

# DigiQ
digiq_key_text = {c: ("Fout/Onjuist = Correct" if DIGIQ_KEY[c] == "fout" else "Juist = Correct") for c in DIGIQ if c in d.columns}
digiq_md = md_section("Codebook: DigiQ") + md_table_from_columns([c for c in DIGIQ if c in d.columns], extra_cols=digiq_key_text)
digiq_md += "\n\n**Scoring:** Correct=1; Incorrect=0; 'Ik weet het niet'=0; 'Ik begrijp de vraag niet'=NaN."

# CDR-items with domain column
def domain_of(c):
    if c in env_cols: return "Environmental"
    if c in soc_cols: return "Social"
    if c in gov_cols: return "Governance"
    return ""
cdr_rows = []
for c in [c for c in CDR_20 if c in d.columns]:
    cdr_rows.append([c, domain_of(c), enriched_label_map.get(c, "")])
cdr_md = md_section("Codebook: CDR-items") + to_markdown_table(cdr_rows, ["Variabele", "Domein (theorie)", "Volledige NL-vraag (QSF-verrijkt of CSV)"])

# RQ1
rq1_cols = [c for c in (RQ1_ENV + RQ1_SOC + RQ1_GOV) if c in d.columns]
rq1_md = md_section("Codebook: RQ1-attributies") + md_table_from_columns(rq1_cols)

# Write codebooks (REMOTE)
rd_write_markdown(pred_md,  OUT_CODEBOOK_PRED_MD)
rd_write_markdown(digiq_md, OUT_CODEBOOK_DIGIQ_MD)
rd_write_markdown(cdr_md,   OUT_CODEBOOK_CDR_MD)
rd_write_markdown(rq1_md,   OUT_CODEBOOK_RQ1_MD)

# --------------------------------------------------------------------------------------------------
# 8) Save kept dataset (REMOTE)
# --------------------------------------------------------------------------------------------------
rd_write_parquet_df(d_keep, OUT_KEEP_DATA, index=False)

# --------------------------------------------------------------------------------------------------
# 9) Reliability (Cronbach’s α) on kept sample
# --------------------------------------------------------------------------------------------------
def cronbach_alpha(df, cols):
    cols = [c for c in cols if c in df.columns]
    sub = df[cols].apply(pd.to_numeric, errors="coerce").dropna()
    n, k = sub.shape
    if n < 2 or k < 2:
        return np.nan, n
    var_sum = sub.var(ddof=1).sum()
    total_var = sub.sum(axis=1).var(ddof=1)
    alpha = (k / (k - 1)) * (1 - var_sum / total_var) if total_var > 0 else np.nan
    return float(alpha), n

alphas = []
for label, cols in [("Environmental", env_cols), ("Social", soc_cols), ("Governance", gov_cols)]:
    if cols:
        a, n = cronbach_alpha(d_keep, cols)
        alphas.append([label, len(cols), n, ("" if pd.isna(a) else f"{a:.3f}")])

# --------------------------------------------------------------------------------------------------
# 10) REPORT — console & notebook pretty-print (REMOTE paths shown)
# --------------------------------------------------------------------------------------------------
hrule("0. Bestandsinfo")
print(f"- CSV (remote): {CSV_PATH}")
print(f"- QSF (remote): {QSF_PATH}\n")

hrule("1. Kolommen gevonden")
def found_str(group):
    have = sum([c in d.columns for c in group])
    return f"gevonden {have}/{len(group)}"
print(f"ENV_CONSC: {found_str(ENV_CONSC)}")
print(f"SOC_CONSC: {found_str(SOC_CONSC)}")
print(f"TRUST: {found_str(TRUST)}")
print(f"DIGIQ: {found_str(DIGIQ)}")
print(f"CDR_ITEMS: {found_str(CDR_20)}")
print(f"RQ1 Env attrib: {found_str(RQ1_ENV)}")
print(f"RQ1 Soc attrib: {found_str(RQ1_SOC)}")
print(f"RQ1 Gov attrib: {found_str(RQ1_GOV)}\n")

hrule("2. Attention checks — resultaat")
n_total = d.shape[0]
n_drop  = len(idx_drop)
n_keep  = d_keep.shape[0]
print(f"N totaal (voor filter): {n_total}")
print(f"N gedropt (attention fail): {n_drop}")
print(f"N behouden (attention pass): {n_keep}")
print(f"Details gedropte cases: {basename(OUT_DROPPED_ATTN)}\n")

# Reliability table
if alphas:
    df_alpha = pd.DataFrame(alphas, columns=["Domein", "Aantal items", "N", "Cronbach’s α"])
    md_table = df_alpha.to_markdown(index=False)
    display(Markdown("### 5️⃣ Betrouwbaarheid — Cronbach’s α per domein (attention-filtered sample)\n"))
    display(Markdown(md_table))
    print(md_table)

# Print codebooks in notebook
display(Markdown("## 📘 Codebook: Predictors")); display(Markdown(pred_md))
display(Markdown("## 💡 Codebook: DigiQ")); display(Markdown(digiq_md))
display(Markdown("## 🌍 Codebook: CDR-items")); display(Markdown(cdr_md))
display(Markdown("## 🧭 Codebook: RQ1-attributies")); display(Markdown(rq1_md))
display(Markdown(demog_md))

# Summary block
summary_md = f"""
### 📊 Samenvatting
| Metric | Aantal |
|:--|--:|
| Totaal aantal respondenten | {n_total} |
| Gedropt (attention fail)   | {n_drop} |
| Behouden (attention pass)  | {n_keep} |

**Bestanden geschreven (remote):**
- Dataset (attention-check pass): `{basename(OUT_KEEP_DATA)}`
- Gedropte cases: `{basename(OUT_DROPPED_ATTN)}`
- Theorie mapping: `{basename(OUT_THEORY_MAP)}`
- Codebooks:
  - `{basename(OUT_CODEBOOK_PRED_MD)}`
  - `{basename(OUT_CODEBOOK_DIGIQ_MD)}`
  - `{basename(OUT_CODEBOOK_CDR_MD)}`
  - `{basename(OUT_CODEBOOK_RQ1_MD)}`
  - `{basename(OUT_CODEBOOK_DEMOG_MD)}`
"""
display(Markdown(summary_md))
print(summary_md)


/tmp/ipykernel_1031282/2328906563.py:88: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(esc)



----------------------------------------------------------------------------------------------------
# 0. Bestandsinfo
----------------------------------------------------------------------------------------------------

- CSV (remote): ASCOR-FMG-14116-CDR (Projectfolder)/data/COS_AI-Tech_October 6, 2025_05.00.csv
- QSF (remote): ASCOR-FMG-14116-CDR (Projectfolder)/data/COS_AI-Tech.qsf


----------------------------------------------------------------------------------------------------
# 1. Kolommen gevonden
----------------------------------------------------------------------------------------------------

ENV_CONSC: gevonden 4/4
SOC_CONSC: gevonden 3/3
TRUST: gevonden 5/5
DIGIQ: gevonden 8/8
CDR_ITEMS: gevonden 20/20
RQ1 Env attrib: gevonden 8/8
RQ1 Soc attrib: gevonden 8/8
RQ1 Gov attrib: gevonden 8/8


----------------------------------------------------------------------------------------------------
# 2. Attention checks — resultaat
--------------------------------------------

### 5️⃣ Betrouwbaarheid — Cronbach’s α per domein (attention-filtered sample)


| Domein        |   Aantal items |    N |   Cronbach’s α |
|:--------------|---------------:|-----:|---------------:|
| Environmental |              3 | 2214 |          0.832 |
| Social        |              9 | 2214 |          0.914 |
| Governance    |              8 | 2214 |          0.944 |

| Domein        |   Aantal items |    N |   Cronbach’s α |
|:--------------|---------------:|-----:|---------------:|
| Environmental |              3 | 2214 |          0.832 |
| Social        |              9 | 2214 |          0.914 |
| Governance    |              8 | 2214 |          0.944 |


...

## 🧭 Codebook: RQ1-attributies



# Codebook: RQ1-attributies
|Variabele|Volledige NL-vraag (QSF-verrijkt of CSV)|
|:---------|:----------------------------------------|
|responsibility attri_1|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Bedrijven die technologie leveren (bedrijven die digitale technologie ontwikkelen)|
|responsibility attri_2|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Bedrijven die technologie gebruiken (bedrijven die technologie gebruiken die door anderen is ontwikkeld, zoals bedrijven die technologieën voor videogesprekken gebruiken)|
|responsibility attri_3|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Werknemers (mensen die voor bedrijven werken, die technologie gebruiken die door anderen is ontwikkeld)|
|responsibility attri_4|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Rijksoverheid en toezichthouders (zoals de Nederlandse overheid)|
|responsibility attri_5|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Europese overheid en regelgevende instanties (zoals de Europese Commissie)|
|responsibility attri_6|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Consumenten (individuen)|
|responsibility attri_7|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Niet-overheidsorganisaties (zoals organisaties die pleiten voor ethische digitale praktijken)|
|responsibility attri_8|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op het milieu (bijv. servers waarop diensten als ChatGPT draaien, hebben veel elektriciteit nodig en dragen bij aan milieuproblemen)? - Nieuwsmedia en journalisten (media en journalisten die verslag uitbrengen over de samenleving)|
|resp_attribut_societ_1|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Bedrijven die technologie leveren (bedrijven die digitale technologie ontwikkelen)|
|resp_attribut_societ_2|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Bedrijven die technologie gebruiken (bedrijven die technologie gebruiken die door anderen is ontwikkeld, zoals bedrijven die technologieën voor videogesprekken gebruiken)|
|resp_attribut_societ_3|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Werknemers (mensen die voor bedrijven werken, die technologie gebruiken die door anderen is ontwikkeld)|
|resp_attribut_societ_4|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Rijksoverheid en toezichthouders (zoals de Nederlandse overheid)|
|resp_attribut_societ_5|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Europese overheid en regelgevende instanties (zoals de Europese Commissie)|
|resp_attribut_societ_6|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Consumenten (individuen)|
|resp_attribut_societ_7|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Niet-overheidsorganisaties (zoals organisaties die pleiten voor ethische digitale praktijken)|
|resp_attribut_societ_8|Wie is er volgens jou verantwoordelijk voor het verminderen van de impact van digitale technologie op de samenleving (bv. kunstmatige intelligentiesystemen zoals ChatGPT en aanbevelingssystemen kunnen discrimineren door de toegang tot bepaalde doelgroepen te beperken)? - Nieuwsmedia en journalisten (media en journalisten die verslag uitbrengen over de samenleving)|
|resp_attribut_regula_1|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Bedrijven die technologie leveren (bedrijven die digitale technologie ontwikkelen)|
|resp_attribut_regula_2|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Bedrijven die technologie gebruiken (bedrijven die technologie gebruiken die door anderen is ontwikkeld, zoals bedrijven die technologieën voor videogesprekken gebruiken)|
|resp_attribut_regula_3|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Werknemers (mensen die voor bedrijven werken, die technologie gebruiken die door anderen is ontwikkeld)|
|resp_attribut_regula_4|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Rijksoverheid en toezichthouders (zoals de Nederlandse overheid)|
|resp_attribut_regula_5|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Europese overheid en regelgevende instanties (zoals de Europese Commissie)|
|resp_attribut_regula_6|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Consumenten (individuen)|
|resp_attribut_regula_7|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Niet-overheidsorganisaties (zoals organisaties die pleiten voor ethische digitale praktijken)|
|resp_attribut_regula_8|Wie is er volgens jou verantwoordelijk voor het reguleren van digitale technologieën en het ethisch gebruik ervan (bijv. het ontwikkelen van richtlijnen die praktische instructies geven over hoe ze moeten worden ontworpen, zoals kunstmatige intelligentiesystemen zoals ChatGPT)? - Nieuwsmedia en journalisten (media en journalisten die verslag uitbrengen over de samenleving)|



# Codebook: Demographics
|Afgeleide variabele|QID (bron)|CSV-kolom|Vraag (bron)|Beschrijving|
|:-------------------|:----------|:---------|:------------|:------------|
|dem_age_band||UserLanguage|Age|Leeftijdscategorie (geordend: 18–24 … 55–65)|
|dem_gender||Gender |Gender|Geslacht|
|dem_employment||Work_status|Work status|Werkstatus|
|dem_education||Education |Education|Hoogst voltooide opleiding|
|dem_political_lr||Pol_orientation|Political L–R|Politieke zelfplaatsing (1–10)|


### 📊 Samenvatting
| Metric | Aantal |
|:--|--:|
| Totaal aantal respondenten | 4155 |
| Gedropt (attention fail)   | 1909 |
| Behouden (attention pass)  | 2246 |

**Bestanden geschreven (remote):**
- Dataset (attention-check pass): `cdr_keep_after_attn.parquet`
- Gedropte cases: `dropped_due_to_attention_checks.csv`
- Theorie mapping: `cdr_theory_map.csv`
- Codebooks:
  - `codebook_predictors.md`
  - `codebook_digiq_key.md`
  - `codebook_cdr_items.md`
  - `codebook_rq1_attribs.md`
  - `codebook_demographics.md`



### 📊 Samenvatting
| Metric | Aantal |
|:--|--:|
| Totaal aantal respondenten | 4155 |
| Gedropt (attention fail)   | 1909 |
| Behouden (attention pass)  | 2246 |

**Bestanden geschreven (remote):**
- Dataset (attention-check pass): `cdr_keep_after_attn.parquet`
- Gedropte cases: `dropped_due_to_attention_checks.csv`
- Theorie mapping: `cdr_theory_map.csv`
- Codebooks:
  - `codebook_predictors.md`
  - `codebook_digiq_key.md`
  - `codebook_cdr_items.md`
  - `codebook_rq1_attribs.md`
  - `codebook_demographics.md`

